In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from pyspark.sql.functions import current_date

In [0]:
df_consumos = (
    spark.read.table("production.transient.pdv_consumos")
    .select(
        F.col("*"),
    )
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_consumos.limit(5))


In [0]:
catalog_table = "production.raw.raw_tb_consumos"

try:
    # --- ETAPA 1: Construir o dicionário de atualização dinamicamente ---
    # É uma boa prática não atualizar a chave usada na condição do merge (consumo_id)
    update_set_dict = {
        col: F.col(f"source.{col}") 
        for col in df_consumos.columns 
        if col not in ["consumo_id", "update_date"]
    }
    
    # Adiciona a coluna de data da atualização ao dicionário
    update_set_dict["update_date"] = current_date()
    
    # --- ETAPA 2: Executar o MERGE com o dicionário pronto ---
    delta_table = DeltaTable.forName(spark, catalog_table)

    print("Tabela Delta encontrada. Executando merge...")
    (delta_table.alias("target")
        .merge(
            df_consumos.alias("source"),
            "target.consumo_id = source.consumo_id"
        )
        .whenMatchedUpdate(
            set=update_set_dict # Passa o dicionário completo e corrigido
        )
        .whenNotMatchedInsertAll()
        .execute()
    )
    
    print(f"Merge na tabela {catalog_table} concluído com sucesso.")

except Exception as e:
    # Se a tabela não existir (ou não for Delta), faz a carga inicial
    # A exceção mais comum na primeira vez é 'DeltaTable' não encontrada
    print(f"Tabela '{catalog_table}' não encontrada ou erro no merge. Realizando carga inicial.")
    #print(f"Detalhe do erro: {e}")
    
    df_consumos.write.format("delta").mode("overwrite").saveAsTable(catalog_table)
    print("Carga inicial concluída com sucesso.")
